# SKAT Enviroment Preperation

## Descpreription
this notebook prepares the neccesary files and scripts to run the SKAT analysis for a given phenotype (in csv file) and plink format genetic files 

In [18]:
import pandas as pd
import pandas_plink as pdp
from ipywidgets import IntProgress
from IPython import display
import os

In [2]:
PROJECT_PATH = "/cs/labs/michall/roeizucker/SKAT_experiments/SetID"
FILE_CONTAINING_PARTICIPANTS = PROJECT_PATH + "/data_participant.csv"
OLD_PHENOTYPE_FILE = '/cs/labs/michall/roeizucker/10krun/runs/48:59/ukbb_dataset.csv'
PHENOTYPE_FILE_NAME = PROJECT_PATH + "/phens.csv"
SET_ID_FILE_MAX_LENGTH = 5000
GENE_FILE_PATH = '/cs/labs/michall/roeizucker/virt_env_install_test/genes_hg19.csv'
# temp covar /cs/labs/michall/roeizucker/phewas_run/runs1/1000:1009/GWAS/covariates.txt

In [32]:
curr_df = pd.read_csv("/cs/labs/michall/roeizucker/SKAT_experiments/SetID/ukbb_dataset_withIIH.csv")


0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
333410    0.0
333411    0.0
333412    0.0
333413    0.0
333414    0.0
Name: IIH_mod, Length: 333415, dtype: float64

In [33]:
curr_df["IIH_mod"].value_counts()

0.0    333242
1.0       173
Name: IIH_mod, dtype: int64

## create basic SetID file 
the SKAT program uses a SetID file that defines which gene contains which variant.

In [4]:
GENES_FILE_PATH = "/cs/usr/roeizucker/my_storage/virt_env_install_test/genes_hg19.csv"

### load genes file
this file is part of the PWAS pipeline, it defines the location and size of each gene

In [5]:
gene_file_dataframe = pd.read_csv(GENES_FILE_PATH)[["uniprot_id","chr","cds_start","cds_end"]]

### load PLINK format genetic files

In [6]:
PLINK_FILE_BASE_PATH = "/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch"

In [7]:
plink_results = []
for i in range(1,23):
    df = pdp.read_plink(PLINK_FILE_BASE_PATH + str(i),verbose=False)[0][["snp","pos"]]
    result = [(x, y) for x, y in zip(df['snp'], df['pos'])]
    plink_results.append((i,result))

In [8]:
print(len(plink_results))

22


In [9]:
print("chr is:",i)
val = pdp.read_plink(PLINK_FILE_BASE_PATH + str(i),verbose=False)


chr is: 22


In [10]:
res = val[2]
res

dask.array<transpose, shape=(15002, 487409), dtype=float32, chunksize=(1024, 1024), chunktype=numpy.ndarray>

### create gene-variant associations

In [11]:
gene_association_result = []

for (chrom, vals) in plink_results:
    reduced_gene_dataframe = gene_file_dataframe[gene_file_dataframe["chr"] == str(chrom)]
    display.clear_output()
    print(chrom)
    counter = 0
    for val in vals:
        pos = val[1]
        name= val[0]
        genes = list(reduced_gene_dataframe[(reduced_gene_dataframe["cds_start"] <= pos) & (reduced_gene_dataframe["cds_end"] >= pos)]["uniprot_id"])
        for gene in genes:
            gene_association_result.append((name,gene))
        counter +=1
        if counter %3000 == 0:
            display.clear_output()
            print(chrom)
            print(counter/len(vals) * 100)
gene_association_result = sorted(gene_association_result,key= lambda x:x[1])

10
24.434941967012826


KeyboardInterrupt: 

In [ ]:
print(len(gene_association_result))

### write variant-gene associations to SetID file

In [ ]:
set_id_file_path = PROJECT_PATH + "/" + "genes.SetID"
counter = 0
lines_seen = set()
counter = 0
with open(set_id_file_path,"w") as file:
    for (variant,gene) in gene_association_result[0:len(gene_association_result) // 2]:
        if counter %10000 == 0:
            display.clear_output()
            print(counter/len(gene_association_result) * 100)
        if f"{gene} {variant}" not in lines_seen:
            file.write(f"{gene} {variant}\n")
            lines_seen.add(f"{gene} {variant}")  
        counter +=1

In [ ]:
PROJECT_PATH

### write variant-gene associations to multiple SetID files

In [ ]:
def get_variants_for_genes(counter,gene_association_result):
    new_counter = counter
    curr_gene = gene_association_result[new_counter][1]    
    gene_vars = []
    while new_counter < len(gene_association_result) and gene_association_result[new_counter][1] == curr_gene:
        gene_vars.append(gene_association_result[new_counter])
        new_counter+=1
        if new_counter %10000 == 0:
            display.clear_output()
            print(new_counter/len(gene_association_result) * 100)
    return gene_vars,new_counter


set_id_file_path = PROJECT_PATH + "/" + "genes.SetID"
counter = 0
lines_seen = set()
counter = 0
curr_file = 0
add_to_curr_file = []
while counter < len(gene_association_result):
    curr_gene = gene_association_result[counter][1]
    add_to_curr_file,counter = get_variants_for_genes(counter,gene_association_result)
    new_counter = counter
    gene_vars = []
    while len(add_to_curr_file) + len(gene_vars) < SET_ID_FILE_MAX_LENGTH and new_counter < len(gene_association_result):
        
        add_to_curr_file.extend(gene_vars)
        counter = new_counter
        if new_counter >= len(gene_association_result):
            print("aaaa")
        gene_vars,new_counter = get_variants_for_genes(counter,gene_association_result)
        
#     set_id_file_path = PROJECT_PATH + "/" + "genes.SetID"
    set_id_file_path = f"{PROJECT_PATH}/genes_{curr_file}.SetID"
    with open(set_id_file_path,"w") as file:
        for (variant,gene) in add_to_curr_file:
            if f"{gene} {variant}" not in lines_seen:
                file.write(f"{gene} {variant}\n")
                lines_seen.add(f"{gene} {variant}")  
    
    curr_file+=1

### create slurm scripts for generating SSD from setID

In [ ]:
# only for a single chromosome:
CHROM = 4
SET_ID_PATH = "/cs/labs/michall/roeizucker/SKAT_experiments/SetID"
R_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{CHROM}.R"
PYTHON_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{CHROM}.py"
CSV_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{CHROM}.csv"
BASH_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{CHROM}.sh"
BASE_PATH = "/cs/labs/michall/roeizucker/IIH"
CURR_BED = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{CHROM}.bed"
CURR_BIM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{CHROM}.bim"
CURR_FAM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{CHROM}.fam"
with open(R_FILE,"w") as file:
        file.write('''require(SKAT)
args = commandArgs(trailingOnly=TRUE)
File.Bed<-args[1]
File.Bim<-args[2]
File.Fam<-args[3]
File.SetID<-args[4]
File.SSD<-args[5]
File.Info<-args[6]
Generate_SSD_SetID(File.Bed, File.Bim, File.Fam, File.SetID, File.SSD, File.Info)
print("Done")
''')


In [ ]:
with open(CSV_FILE,"w") as file:
    file.write("bed,bim,fam,SetID,SSD,info\n")
    for i in range(curr_file):
        file.write(f"{CURR_BED},{CURR_BIM},{CURR_FAM},{SET_ID_PATH}/genes_{i}.SetID,{BASE_PATH}/genes_{CHROM}_{i}.SSD,{BASE_PATH}/genes_{CHROM}_{i}.SSD.info\n")
    

In [ ]:
import pandas as pd
import sys
import os
df = pd.read_csv(CSV_FILE)
line = df.iloc[int(sys.argv[1]),]
# line = df.iloc[3,]
os.system('/cs/cbio/netanel/lib/R4/bin/Rscript ' + R_FILE + ' ' + line.bed + ' ' + line.bim + ' ' + line.fam + ' ' + line.SetID + ' ' + line.SSD + ' ' + line.info)

In [ ]:
with open(PYTHON_FILE,"w") as file:
    file.write("import pandas as pd\n")
    file.write("import sys\n")
    file.write("import os\n")
    file.write("df = pd.read_csv('"+CSV_FILE+"')\n")
    file.write("line = df.iloc[int(sys.argv[1]),]\n")
    file.write("os.system('/cs/cbio/netanel/lib/R4/bin/Rscript " + R_FILE + " ' + line.bed + ' ' + line.bim + ' ' + line.fam + ' ' + line.SetID + ' ' + line.SSD + ' ' + line.info)\n")
    file.write("\n")
    


In [ ]:
with open(BASH_FILE,"w") as file:
    file.write(f"python {PYTHON_FILE} $SLURM_ARRAY_TASK_ID\n")


In [ ]:
!chmod 744 {BASH_FILE}

In [ ]:
!echo {BASH_FILE}

### create SETId files for multiple chromosomes

In [ ]:
for chrom in range(0,23):
    SET_ID_PATH = "/cs/labs/michall/roeizucker/SKAT_experiments/SetID"
    R_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{chrom}.R"
    PYTHON_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{chrom}.py"
    CSV_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{chrom}.csv"
    BASH_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{chrom}.sh"
    BASE_PATH = "/cs/labs/michall/roeizucker/IIH"
    CURR_BED = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{chrom}.bed"
    CURR_BIM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{chrom}.bim"
    CURR_FAM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{chrom}.fam"
    with open(R_FILE,"w") as file:
        file.write('''require(SKAT)
    args = commandArgs(trailingOnly=TRUE)
    File.Bed<-args[1]
    File.Bim<-args[2]
    File.Fam<-args[3]
    File.SetID<-args[4]
    File.SSD<-args[5]
    File.Info<-args[6]
    Generate_SSD_SetID(File.Bed, File.Bim, File.Fam, File.SetID, File.SSD, File.Info)
    print("abcd")
    ''')
    with open(CSV_FILE,"w") as file:
        file.write("bed,bim,fam,SetID,SSD,info\n")
        for i in range(curr_file):
            file.write(f"{CURR_BED},{CURR_BIM},{CURR_FAM},{SET_ID_PATH}/genes_{i}.SetID,{BASE_PATH}/genes_{chrom}_{i}.SSD,{BASE_PATH}/genes_{chrom}_{i}.SSD.info\n")
    with open(PYTHON_FILE,"w") as file:
        file.write("import pandas as pd\n")
        file.write("import sys\n")
        file.write("import os\n")
        file.write("df = pd.read_csv('"+CSV_FILE+"')\n")
        file.write("line = df.iloc[int(sys.argv[1]),]\n")
        file.write("os.system('/cs/cbio/netanel/lib/R4/bin/Rscript " + R_FILE + " ' + line.bed + ' ' + line.bim + ' ' + line.fam + ' ' + line.SetID + ' ' + line.SSD + ' ' + line.info)\n")
        file.write("\n")
    
    with open(BASH_FILE,"w") as file:
        file.write(f"python {PYTHON_FILE} $SLURM_ARRAY_TASK_ID\n")

    !chmod 744 {BASH_FILE}
    !echo {BASH_FILE}

In [ ]:
for chrom in range(0,5):
    BASH_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{chrom}.sh"
    print(f'sbatch --array=0-150 --mem=15g -c1 --time=1-0 --killable --requeue --wrap="{BASH_FILE}"')
    print("sleep 1m")
    print(f"echo {BASH_FILE} running")
#     print("sleep 24m")

### create output files from SSD for all chromosomes

In [ ]:
for chrom in range(1,23):
    SET_ID_PATH = "/cs/labs/michall/roeizucker/IIH"
    CSV_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{chrom}.csv"
    BASE_PATH = "/cs/labs/michall/roeizucker/IIH"
    CURR_BED = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{chrom}.bed"
    CURR_BIM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{chrom}.bim"
    CURR_FAM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{chrom}.fam"
    
    R_FILE = f"/cs/labs/michall/roeizucker/hackton_meno/scripts/create_out_{chrom}.R"
    PYTHON_FILE = f"/cs/labs/michall/roeizucker/hackton_meno/scripts/create_out_{chrom}.py"
    # CSV_FILE = "/cs/usr/roeizucker/Desktop/temp/SKAT_delete_me/scripts/create_SSD.csv"
    BASH_FILE = f"/cs/labs/michall/roeizucker/hackton_meno/scripts/create_out_{chrom}.sh"
    with open(R_FILE,"w") as file:
        file.write('''require(SKAT)
args = commandArgs(trailingOnly=TRUE)
File.Bed<-args[1]
File.Bim<-args[2]
File.Fam<-args[3]
File.SetID<-args[4]
File.SSD<-args[5]
File.Info<-args[6]
File.Cov<-"/cs/labs/michall/roeizucker/phewas_run/runs1/1000:1009/GWAS/covariates.txt"

FAM_Cov<-Read_Plink_FAM_Cov(File.Fam, File.Cov, Is.binary=FALSE, cov_header=FALSE)

X2 = FAM_Cov$Sex
X1 = FAM_Cov$COV1

SSD.INFO<-Open_SSD(File.SSD, File.Info)

test_phenotypes = read.csv(file = "/cs/usr/roeizucker/Desktop/temp/SKAT_delete_me/phens.csv")
obj<-SKAT_Null_Model(test_phenotypes$IIH_mod ~ data.matrix(FAM_Cov[,7:14]), out_type="C")

SSD.INFO<-Open_SSD(File.SSD, File.Info)
out<-SKAT.SSD.All(SSD.INFO, obj)

write.table(out$results,args[7])
''')
    with open(PYTHON_FILE,"w") as file:
        file.write("import pandas as pd\n")
        file.write("import sys\n")
        file.write("import os\n")
        file.write("df = pd.read_csv('"+CSV_FILE+"')\n")
        file.write("out_file = '" + BASE_PATH + "/out_'+  str(sys.argv[1]) +'_"+str(chrom)+".csv'\n")
        file.write("line = df.iloc[int(sys.argv[1]),]\n")
        file.write("os.system('/cs/cbio/netanel/lib/R4/bin/Rscript " + R_FILE + " ' + line.bed + ' ' + line.bim + ' ' + line.fam + ' ' + line.SetID + ' ' + line.SSD + ' ' + line.info+ ' ' + out_file)\n")
        file.write("\n")
    with open(BASH_FILE,"w") as file:
        file.write(f"python {PYTHON_FILE} $SLURM_ARRAY_TASK_ID\n")
    !chmod 744 {BASH_FILE}
    !echo {BASH_FILE}

In [ ]:
for chrom in range(0,23):
    BASH_FILE = f"/cs/labs/michall/roeizucker/hackton_meno/scripts/create_out_{chrom}.sh"
    print(f'sbatch --array=0-150 --mem=15g -c1 --time=1-0 --killable --requeue --wrap="{BASH_FILE}"')
    print("sleep 1m")
    print(f"echo {BASH_FILE} running")
    print("sleep 5m")


### create output files from SSD

In [ ]:
CHROM = 2
# SET_ID_PATH = "/cs/usr/roeizucker/Desktop/temp/SKAT_delete_me"
CSV_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_SSD_{CHROM}.csv"
BASE_PATH = "/cs/labs/michall/roeizucker/IIH"
CURR_BED = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{CHROM}.bed"
CURR_BIM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{CHROM}.bim"
CURR_FAM = f"/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{CHROM}.fam"

R_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_out_{CHROM}.R"
PYTHON_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_out_{CHROM}.py"
# CSV_FILE = "/cs/usr/roeizucker/Desktop/temp/SKAT_delete_me/scripts/create_SSD.csv"
BASH_FILE = f"/cs/labs/michall/roeizucker/IIH/scripts/create_out_{CHROM}.sh"
with open(R_FILE,"w") as file:
    file.write('''require(SKAT)
args = commandArgs(trailingOnly=TRUE)
File.Bed<-args[1]
File.Bim<-args[2]
File.Fam<-args[3]
File.SetID<-args[4]
File.SSD<-args[5]
File.Info<-args[6]

SSD.INFO<-Open_SSD(File.SSD, File.Info)

test_phenotypes = read.csv(file = "/cs/usr/roeizucker/Desktop/temp/SKAT_delete_me/phens.csv")
obj<-SKAT_Null_Model(test_phenotypes$IIH_mod ~ 1, out_type="C")
SSD.INFO<-Open_SSD(File.SSD, File.Info)
out<-SKAT.SSD.All(SSD.INFO, obj)
print(args[7])
write.table(out$results,args[7])
''')

In [ ]:
with open(PYTHON_FILE,"w") as file:
    file.write("import pandas as pd\n")
    file.write("import sys\n")
    file.write("import os\n")
    file.write("df = pd.read_csv('"+CSV_FILE+"')\n")
    file.write("out_file = '" + BASE_PATH + "/out_'+  str(sys.argv[1]) +'_"+str(CHROM)+".csv'\n")
    file.write("line = df.iloc[int(sys.argv[1]),]\n")
    file.write("os.system('/cs/cbio/netanel/lib/R4/bin/Rscript " + R_FILE + " ' + line.bed + ' ' + line.bim + ' ' + line.fam + ' ' + line.SetID + ' ' + line.SSD + ' ' + line.info+ ' ' + out_file)\n")
    file.write("\n")


In [ ]:
with open(BASH_FILE,"w") as file:
    file.write(f"python {PYTHON_FILE} $SLURM_ARRAY_TASK_ID\n")


In [ ]:
!chmod 744 {BASH_FILE}

In [ ]:
!ls "/cs/labs/michall/roeizucker/IIH/scripts/create_out_2.sh"

In [ ]:
!ls /cs/usr/roeizucker/Desktop/temp/SKAT_delete_me/scripts/

In [ ]:
print(counter)

## create new phenotype file

In [29]:
participants_dataframe = pd.read_csv(FILE_CONTAINING_PARTICIPANTS)
participants_dataframe
participants_dataframe["IIH_mod"] = 1
participants_dataframe["eid2"] = participants_dataframe["eid"]
participants_dataframe.set_index("eid2",inplace=True)
participants_dataframe
# print(len(old_phenotype_dataframe))
old_phenotype_dataframe = pd.read_csv(OLD_PHENOTYPE_FILE)
old_phenotype_dataframe["eid2"] = old_phenotype_dataframe["eid"]
old_phenotype_dataframe.set_index("eid2",inplace=True)
old_phenotype_dataframe["IIH_mod"] = participants_dataframe["IIH_mod"]
old_phenotype_dataframe["IIH_mod"] = old_phenotype_dataframe["IIH_mod"].fillna(0)
print(len(old_phenotype_dataframe))


333415


In [30]:
old_phenotype_dataframe["IIH_mod"].value_counts()

0.0    333242
1.0       173
Name: IIH_mod, dtype: int64

In [17]:
old_phenotype_dataframe[(old_phenotype_dataframe["IIH_mod"] == 1) & (old_phenotype_dataframe["sex"] == 0)]

,sample_index,eid,F329,F329_F,F329_M,H26,H26_F,H26_M,H269,H269_F,...,batch_UKBiLEVEAX_b10,batch_UKBiLEVEAX_b11,batch_UKBiLEVEAX_b3,batch_UKBiLEVEAX_b4,batch_UKBiLEVEAX_b5,batch_UKBiLEVEAX_b6,batch_UKBiLEVEAX_b7,batch_UKBiLEVEAX_b8,batch_UKBiLEVEAX_b9,IIH_mod
eid2,,,,,,,,,,,,,,,,,,,,,
5580060,4625,5580060,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1.0
4357780,5332,4357780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4994919,7895,4994919,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1.0
4556922,8337,4556922,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1.0
1243359,8460,1243359,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4722410,482630,4722410,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1.0
5899992,484212,5899992,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1.0
1101465,488092,1101465,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,1,1.0


In [19]:
old_phenotype_dataframe.to_csv(os.path.join(PROJECT_PATH,"ukbb_dataset_withIIH.csv"))

In [26]:
os.path.join(PROJECT_PATH,"ukbb_dataset_withIIH.csv")

'/cs/labs/michall/roeizucker/SKAT_experiments/SetID/ukbb_dataset_withIIH.csv'

### sort new dataframe - change, only for presentation

In [20]:
TEST_BIM_PATH = "/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered.bim"
TEST_BED_PATH = "/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered.bed"
TEST_FAM_PATH = "/cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered.fam"
PLINK_PATH = "/cs/usr/nadavb/third_party/plink2"


In [ ]:
a = pdp.read_plink1_bin(TEST_BED_PATH,TEST_BIM_PATH,TEST_FAM_PATH,verbose=False)


In [ ]:
eid_vals_in_files =  list(a['iid'].values)
eid_vals_in_files_set = set(eid_vals_in_files)


In [ ]:
old_phenotype_dataframe = old_phenotype_dataframe[old_phenotype_dataframe['eid'].isin(eid_vals_in_files)]


In [ ]:
print(len(old_phenotype_dataframe))
print(len(eid_vals_in_files))


In [ ]:
old_phenotype_dataframe['sort_key'] = old_phenotype_dataframe['eid'].astype("str").apply(lambda x: eid_vals_in_files.index(x) if x in eid_vals_in_files_set else float('inf'))


In [ ]:
old_phenotype_dataframe = old_phenotype_dataframe.sort_values(by='sort_key')


In [ ]:
old_phenotype_dataframe[["eid","IIH_mod"]].to_csv(PHENOTYPE_FILE_NAME)

In [ ]:
print(PHENOTYPE_FILE_NAME)

In [ ]:
old_phenotype_dataframe

## create compatible Plink and phenotype files
for SKAT to work the phenotype and genotype files need to be the same length, and have the same order. Participants that appear in only one dataset need to be removed.

In [ ]:
BASE_PLINK_FORMAT_PATH = "/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch"
PLINK_PATH = "/cs/usr/nadavb/third_party/plink2"


In [ ]:
for i in range(1,2):
    bim_path = BASE_PLINK_FORMAT_PATH + str(i) + ".bim"
    fam_path = BASE_PLINK_FORMAT_PATH + str(i) + ".fam"
    bed_path = BASE_PLINK_FORMAT_PATH + str(i) + ".bed"
    ! {PLINK_PATH} --bim {bim_path} --fam {fam_path} --bed {bed_path} --keep-fam /cs/labs/michall/roeizucker/SKAT_experiments/I10_eid_notNA.txt --make-bed --out /cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{i}

In [ ]:
! {PLINK_PATH} --bim {TEST_BIM_PATH} --fam {TEST_FAM_PATH} --bed {TEST_BED_PATH} --keep-fam /cs/labs/michall/roeizucker/SKAT_experiments/I10_eid_notNA.txt --make-bed --out /cs/labs/michall/roeizucker/SKAT_experiments/mydata_filtered_{i}

## Create phenotype files
Phenotype files must be the same order and lenght as the genotype files

In [ ]:
BASIC_PHENOTYPE_FILE = "/cs/usr/roeizucker/Desktop/temp/SKAT_delete_me/phens.csv"

In [ ]:
basic_phenotype_df = pd.read_csv(BASIC_PHENOTYPE_FILE) # we will use this DF to determine order and length of phenotypes
other_phenotype_df = pd.read_csv("/cs/labs/michall/roeizucker/ukb_files/menapause_with_outcomes.csv")

In [ ]:
other_phenotype_df["eid"] = other_phenotype_df["Participant ID"]


In [ ]:
other_phenotype_df["eid2"] = other_phenotype_df["eid"]
other_phenotype_df = other_phenotype_df[other_phenotype_df['eid2'].isin(list(basic_phenotype_df["eid2"]))]

In [ ]:
len(basic_phenotype_df)

In [ ]:
basic_phenotype_df.set_index(['eid2'], inplace=True)
other_phenotype_df.set_index(['eid2'], inplace=True)

In [ ]:
other_phenotype_df["osteoporosis"] = other_phenotype_df["M81.9 Osteoporosis, unspecified_Label"]
other_phenotype_df["fatty_liver"] = other_phenotype_df["K76.0 Fatty (change of) liver, not elsewhere classified_Label"]
basic_phenotype_df["osteoporosis"] = other_phenotype_df["osteoporosis"]
basic_phenotype_df["fatty_liver"] = other_phenotype_df["fatty_liver"]

In [ ]:
basic_phenotype_df[basic_phenotype_df["fatty_liver"] == -1] = 0
basic_phenotype_df[basic_phenotype_df["osteoporosis"] == -1] = 0

In [ ]:
basic_phenotype_df["osteoporosis"] = basic_phenotype_df["osteoporosis"].fillna(0)
basic_phenotype_df["fatty_liver"] = basic_phenotype_df["fatty_liver"].fillna(0)

In [ ]:
basic_phenotype_df.to_csv("/cs/labs/michall/roeizucker/hackton_meno/labels_1.csv",index=False)

In [ ]:
print(PROJECT_PATH)

In [ ]:
# find files for a specific gene
import os, glob
path = "/cs/labs/michall/roeizucker/IIH"
for filename in glob.glob(os.path.join(path, '*.SetID')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
        val = f.read()
        if "Q9UNW9" in val:
            print(filename)
#             print(val)

In [ ]:
# find files for a specific gene
import os, glob
path = "~"
for filename in glob.glob(os.path.join(path, '*.csv')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
        if "Q9UN" in f.read():
            print(filename)

In [ ]:
# create GWAS phenotype file:
# f'sbatch --array=0-150 --mem=15g -c1 --time=1-0 --killable --requeue --wrap="{BASH_FILE}"')
for i in range(1,23):
    inner_command = (f"/cs/usr/nadavb/third_party/plink2 --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{i}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{i}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{i}.fam --pheno /cs/labs/michall/roeizucker/IIH/GWAS/phenotypes/iih_mod.txt --covar /cs/labs/michall/roeizucker/amos/continuous_run/GWAS/covariates.txt  --out  /cs/labs/michall/roeizucker/IIH/GWAS/results/iih_mod_chr{i} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
    print(f'sbatch --mem=15g -c1 --time=1-0 --killable --requeue --wrap="{inner_command}"')

In [ ]:
# small covariates /cs/labs/michall/roeizucker/IIH/GWAS/covariates.txt 
# create GWAS phenotype file:
# f'sbatch --array=0-150 --mem=15g -c1 --time=1-0 --killable --requeue --wrap="{BASH_FILE}"')
for i in range(1,23):
    inner_command = (f"/cs/usr/nadavb/third_party/plink2 --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{i}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{i}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{i}.fam --pheno /cs/labs/michall/roeizucker/IIH/GWAS/phenotypes/iih_mod.txt --covar /cs/labs/michall/roeizucker/IIH/GWAS/covariates.txt  --out  /cs/labs/michall/roeizucker/IIH/GWAS/results/iih_mod_small_cov_chr{i} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
    print(f'sbatch --mem=15g -c1 --time=1-0 --killable --requeue --wrap="{inner_command}"')

In [23]:
# create smaller covariats file:
path = '/cs/labs/michall/roeizucker/10krun/runs/0:11'
PHEN_PATH = path + "/ukbb_dataset.csv"
GWAS_PATH = "/cs/labs/michall/roeizucker/IIH/GWAS"
cancer_dataset = pd.read_csv(PHEN_PATH)
# FAM_Cov[,7:14]

covariate_cols = list(cancer_dataset)[15:23]
covariate_values = cancer_dataset[['eid', 'eid'] + [covariate_col for covariate_col in covariate_cols if \
        covariate_col != 'const']].dropna()
covariate_values.to_csv(f"{GWAS_PATH}/covariates.txt", header = False, index = False, sep = '\t')
print('Done.')

Done.


In [24]:
f"{GWAS_PATH}/covariates.txt"

'/cs/labs/michall/roeizucker/IIH/GWAS/covariates.txt'

In [22]:
GWAS_PATH

'/cs/labs/michall/roeizucker/IIH/GWAS'

In [ ]:
# combine plink results
all_dfs = []
for i in range(1,23):
    df = pd.read_csv(f"/cs/labs/michall/roeizucker/IIH/GWAS/results/iih_mod_chr{i}.PHENO1.glm.logistic", sep="\t")
    all_dfs.append(df)



In [ ]:
# combine small covariates plink results
all_small_covar_dfs = []
for i in range(1,23):
    df = pd.read_csv(f"/cs/labs/michall/roeizucker/IIH/GWAS/results/iih_mod_small_cov_chr{i}.PHENO1.glm.logistic", sep="\t")
    all_dfs.append(df)

# save file
small_cover_result = pd.concat(all_dfs)
small_cover_result = small_cover_result.reset_index()


In [ ]:
result = pd.concat(all_dfs)
result = result.reset_index()


In [ ]:
result

In [ ]:
# add gene to result:
def add_genes_for_snps(df,genes_csv):
    sig_genes = set()
    for i in range(len(df)):
        if i % 10000 == 0:
            print(i/len(df) * 100)
        row = df.loc[i]
        curr = (genes_csv.query("chr == '" + str(row["#CHROM"]) + "' and cds_start <= " + str(row["POS"]) + "and cds_end >= " + str(row["POS"])))
        if len(list(curr["uniprot_id"]))> 0:
            df.loc[i,"GENE"] = list(curr["uniprot_id"])[0]
#         row["GENE"] = list(curr["uniprot_id"])[0]
#         sig_genes.add(list(curr["uniprot_id"])[0])
    return sig_genes
genes_csv = pd.read_csv(GENE_FILE_PATH)
small_cover_result["GENE"] = None
# add_genes_for_snps(result,genes_csv)
add_genes_for_snps(small_cover_result,genes_csv)

# result.loc[3,"GENE"] = "bob" small_cover_result
# print(result.loc[3])

In [ ]:
result.to_csv("/cs/labs/michall/roeizucker/IIH/GWAS/results/iih_mod_gwas.csv")

In [ ]:
small_cover_result

In [ ]:
result

In [ ]:
import pandas as pd
import dash_bio

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/manhattan_data.csv')

result["CHR"] = result["#CHROM"]
result["BP"] = result["POS"]
result["SNP"] = result["ID"]

dash_bio.ManhattanPlot(
    dataframe=result,
)
